In [2]:
# import lib
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt

np.random.seed(78)
tf.set_random_seed(678)

In [92]:
# declare
num_epoch    = 10000
ground_truth = np.random.uniform(0,1,10)
expected     = np.zeros(10)
pull_count   = np.zeros(10) 
print(ground_truth)
print(expected)
print(pull_count)

[0.81026535 0.4242292  0.82882666 0.00941958 0.08615632 0.46276939
 0.11708243 0.48577209 0.43982427 0.16147865]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [93]:
# train 
num_epoch = 1000000
for iter in range(num_epoch):
    current_action = np.random.randint(10)
    pull_count[current_action] = pull_count[current_action] + 1
    if np.random.uniform(0,1) < ground_truth[current_action]: current_reward = 1
    else: current_reward = 0
    expected[current_action] = expected[current_action] + (1/pull_count[current_action]) * (current_reward-expected[current_action])
    
print(np.around(ground_truth,3))
print(np.around(expected,3))
print(np.around(pull_count,3))

[0.81  0.424 0.829 0.009 0.086 0.463 0.117 0.486 0.44  0.161]
[0.812 0.427 0.828 0.01  0.086 0.463 0.116 0.489 0.439 0.161]
[ 99889.  99828. 100154. 100062.  99850. 100043. 100048. 100294.  99927.
  99905.]


In [102]:
# declare
expected     = np.zeros(10)
pull_count   = np.zeros(10) 
print(ground_truth)
print(expected)
print(pull_count)

[0.81026535 0.4242292  0.82882666 0.00941958 0.08615632 0.46276939
 0.11708243 0.48577209 0.43982427 0.16147865]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [127]:
# train 
import threading

def findvalue():
    global expected,pull_count
    current_action = np.random.randint(10)
    pull_count[current_action] = pull_count[current_action] + 1
    if np.random.uniform(0,1) < ground_truth[current_action]: current_reward = 1
    else: current_reward = 0
    expected[current_action] = expected[current_action] + (1/pull_count[current_action]) * (current_reward-expected[current_action])
  
jobs = []
for i in range(0, 2):
    thread = threading.Thread(target=findvalue())
    jobs.append(thread)
    

for x in jobs:
    x.start()

for x in jobs:
    x.join()

print(np.around(ground_truth,3))
print(np.around(expected,3))
print(np.around(pull_count,3))

[0.81  0.424 0.829 0.009 0.086 0.463 0.117 0.486 0.44  0.161]
[0.875 0.333 1.    0.    0.    0.333 0.125 0.5   0.111 0.5  ]
[8. 3. 1. 4. 6. 3. 8. 6. 9. 2.]


# Reference 
1. Solving the Multi-Armed Bandit Problem – Towards Data Science. (2017). Towards Data Science. Retrieved 24 December 2018, from https://towardsdatascience.com/solving-the-multi-armed-bandit-problem-b72de40db97c
2. 